In [26]:
import os

os.chdir("C:/Users/anton/Desktop/University/AI_Lab/Git_repo")

In [27]:
import pandas as pd
import torch
from fix_data import add_label_T
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score
from torch import nn
from torch import optim

In [213]:
df = pd.read_csv("data/SmartSeq/MCF7_SmartS_Filtered_Normalised_3000_Data_train.txt", sep=" ")
df = add_label_T(df, dropname=True)
df

,CYP1B1,CYP1B1-AS1,CYP1A1,NDRG1,DDIT4,PFKFB3,HK2,AREG,MYBL2,ADM,...,DNAI7,MAFG,LZTR1,BCO2,GRIK5,SLC25A27,DENND5A,CDK5R1,FAM13A-AS1,label
0,343,140,0,0,386,75,0,0,476,0,...,0,17,59,0,0,0,51,0,0,1
1,131,59,0,1,289,42,0,856,1586,0,...,0,1,25,0,0,0,34,0,0,1
2,452,203,0,0,0,0,0,0,775,0,...,0,23,0,0,0,0,0,0,0,1
3,27,7,0,0,288,214,15,242,1191,0,...,0,3,10,0,0,0,4,0,0,1
4,5817,2669,0,654,2484,1603,748,436,0,0,...,0,63,0,0,0,0,60,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,196,102,1,243,266,278,78,1,199,0,...,0,1,0,0,0,0,45,19,0,1
246,504,238,0,62,417,932,275,0,33,0,...,0,0,0,0,0,0,0,0,0,1
247,34565,13717,11274,1263,4256,5663,701,0,0,0,...,0,45,0,0,0,0,140,0,32,0
248,20024,7835,563,925,12733,2187,1141,0,20,81,...,0,57,8,0,0,0,7,0,11,0


In [95]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("label", axis=1), df["label"], test_size=0.2)
X, y = df.drop("label", axis=1), df["label"]

In [32]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = clf.score(X_test, y_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.972972972972973
Precision: 1.0
Recall: 0.95
F1 Score: 0.9743589743589743


In [33]:
scores = cross_val_score(clf, X, y, cv=10)
print(scores.mean())

0.9777777777777779


In [205]:
class NeuralNetwrok(nn.Module):
    def __init__(self, D, H):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Linear(D, H),
            nn.ReLU(),
            nn.Linear(H, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.stack(x)
    

def train_loop(X, y, eta, epochs, H):
    N, D = X.shape
    
    model = NeuralNetwrok(D, H)
    optimizer = torch.optim.Adam(model.parameters(), lr=eta)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        y_pred = model(X)
        loss = criterion(y_pred, y)
        if epoch % 1000 == 0:
            print(epoch, loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    return model

In [216]:
X_train, X_test, y_train, y_test = train_test_split(df.drop("label", axis=1), df["label"], test_size=0.5)
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = y_test.tolist()

model = train_loop(X_train, y_train, 0.0001, 5000, 16)

with torch.no_grad():
    y_pred = [int(x) for x in model(X_test).view(-1).tolist()]


right = 0
N = len(y_test)
for i in range(N):
    if y_pred[i] == y_test[i]:
        right += 1
accuracy = right / N
print("Accuracy:", accuracy)


0 52.917686462402344
1000 52.79999923706055
2000 52.79999923706055
3000 52.79999923706055
4000 52.79999923706055
Accuracy: 0.536


[0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
